In [1]:
import numpy as np
import pandas as pd 
import re 
import json
import timeit
import networkx as nx
import matplotlib.pyplot as plt

In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

## Read patents

In [2]:
ids = pd.read_csv(r"../patents/gp-query2-plus.csv",index_col='code',skipinitialspace=True)
print(ids.isnull().sum())
print('length is',len(ids))

id                                0
title                             0
assignee                         15
inventor/author                 383
priority date                   324
filing/creation date              6
publication date                  1
grant date                    13623
result link                       0
representative figure link    10002
code.1                            0
citations                         0
abstract                        796
class                          3384
dtype: int64
length is 22573


In [3]:
ids = ids[ids.abstract.notna()]
ids["class"].fillna(value="NA",inplace = True)
ids['class'] = ids['class'].astype('str').apply(lambda x: x[:3] if len(x) >= 3 else 'NA')
ids["class"] = ids["class"].apply(lambda x: 'NA' if x.find('G06')<0 and x.find('H04')<0 else x)
ids['abstract'] = ids['abstract'].str.replace(r'[^\x00-\x7F]+', '')

print(ids.isnull().sum())
ids['class'].value_counts()

id                                0
title                             0
assignee                          5
inventor/author                 369
priority date                   315
filing/creation date              1
publication date                  0
grant date                    13565
result link                       0
representative figure link     9736
code.1                            0
citations                         0
abstract                          0
class                             0
dtype: int64


G06    9582
NA     6895
H04    5300
Name: class, dtype: int64

## Read articles

In [4]:
arts = pd.read_csv(r"../papers/scopus-query2.csv",skipinitialspace=True)
arts

,Year,Cited by,Link,Abstract,EID
0,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Generally, the decision rule for classifying u...",2-s2.0-85099418104
1,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,This survey explores how Deep Learning has bat...,2-s2.0-85099378785
2,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,The way electorates were influenced to vote fo...,2-s2.0-85094573545
3,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,This study is informed by two research gaps. O...,2-s2.0-85100650454
4,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Over recent years, the development of online s...",2-s2.0-85101111750
...,...,...,...,...,...
7403,2020,NaN,https://www.scopus.com/inward/record.uri?eid=2...,The use of Twitter by newspapers is widespread...,2-s2.0-85096716723
7404,2020,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"In the present technological era, healthcare p...",2-s2.0-85090341003
7405,2020,1,https://www.scopus.com/inward/record.uri?eid=2...,Internet of Things (IoT) devices have connecte...,2-s2.0-85081137773
7406,2020,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Subject Matter/Topic.Today, the issues of lega...",2-s2.0-85081685033


In [5]:
arts = arts[arts['EID'].str.find('2-s2.')>=0] #deleting this strange lines
arts['EID'] = arts['EID'].apply(lambda x: x.replace('2-s2.0-',''))
arts.set_index(arts['EID'],inplace=True)
arts

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Cited by,Link,Abstract,EID
EID,,,,,
85099418104,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Generally, the decision rule for classifying u...",85099418104
85099378785,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,This survey explores how Deep Learning has bat...,85099378785
85094573545,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,The way electorates were influenced to vote fo...,85094573545
85100650454,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,This study is informed by two research gaps. O...,85100650454
85101111750,2021,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Over recent years, the development of online s...",85101111750
...,...,...,...,...,...
85096716723,2020,NaN,https://www.scopus.com/inward/record.uri?eid=2...,The use of Twitter by newspapers is widespread...,85096716723
85090341003,2020,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"In the present technological era, healthcare p...",85090341003
85081137773,2020,1,https://www.scopus.com/inward/record.uri?eid=2...,Internet of Things (IoT) devices have connecte...,85081137773


# Topic Modeling

In [13]:
x_train = [abstract for abstract in ids.abstract]
x_test = [abstract for abstract in arts.Abstract]

print(len(x_train),len(x_test))
#x_train = x_train.extend(x_test)






21777 7405


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

it_stopwords = stopwords.words('english')

tf_vectorizer = CountVectorizer(stop_words=it_stopwords, max_df=0.5, min_df=5,max_features = 1000, ngram_range=(1,2))
tf = tf_vectorizer.fit_transform(x_train)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 8

lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=1)
lda.fit(tf)


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [9]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [16]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0: data, network, system, security, based, model, event, traffic, learning, set, risk, machine, analysis, attack, detection, time, module, using, machine learning, behavior
Topic 1: file, code, malicious, software, program, message, malware, application, method, domain, document, files, name, call, web, computer, detection, based, whether, system
Topic 2: one, least, least one, data, plurality, includes, method, content, based, input, set, associated, system, first, configured, processor, processing, memory, including, determining
Topic 3: information, user, invention, method, terminal, system, data, according, present, present invention, management, module, service, server, mobile, comprises, time, provides, video, platform
Topic 4: first, data, second, device, network, communication, server, node, key, system, authentication, electronic, information, method, transaction, storage, user, using, block, includes
Topic 5: control, system, unit, module, device, vehicle, connected, si

In [10]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0: malware, detection, learning, based, machine, malicious, machine learning, features, using, model, network, classification, neural, analysis, android, feature, accuracy, method, system, proposed
Topic 1: device, data, network, may, computer, one, system, first, user, based, computing, security, application, systems, include, method, methods, second, associated, devices
Topic 2: information, method, data, invention, according, target, model, network, comprises, present, based, present invention, corresponding, steps, first, result, value, node, discloses, terminal
Topic 3: social, data, information, media, security, analysis, paper, based, using, cyber, social media, research, use, used, systems, also, intelligence, results, election, networks
Topic 4: image, control, device, unit, video, system, signal, invention, recognition, display, sensor, object, power, face, detection, camera, connected, method, area, monitoring
Topic 5: user, information, system, content, vehicle, serve

In [17]:
doc_topics = lda.transform(tf)
doc_topics

array([[0.00272057, 0.00271934, 0.00272074, ..., 0.00272198, 0.92976973,
        0.00271961],
       [0.00178654, 0.00178676, 0.00178971, ..., 0.0017882 , 0.58325176,
        0.00178751],
       [0.00432007, 0.00432125, 0.00431384, ..., 0.00431411, 0.00431644,
        0.69105733],
       ...,
       [0.16776644, 0.00164661, 0.0016458 , ..., 0.27879357, 0.18347814,
        0.0016461 ],
       [0.00115891, 0.34693712, 0.10014695, ..., 0.29690568, 0.00115967,
        0.00115982],
       [0.00266244, 0.00266179, 0.33077775, ..., 0.50136357, 0.15454628,
        0.00266233]])

In [19]:
ids['topic'] = np.argmax(doc_topics,axis=1)

In [20]:
ids['topic'].value_counts()

7    3910
5    3259
6    2999
2    2963
0    2758
3    2694
4    2300
1     894
Name: topic, dtype: int64

In [18]:
tf2 = tf_vectorizer.transform(x_test)
arts_topics = lda.transform(tf2)


array([[0.37654146, 0.00125167, 0.00125226, ..., 0.07980312, 0.48750139,
        0.05114678],
       [0.29859618, 0.00183989, 0.00184113, ..., 0.00183941, 0.48215709,
        0.14533458],
       [0.17384004, 0.00260665, 0.00260662, ..., 0.10637982, 0.0026095 ,
        0.47312797],
       ...,
       [0.54653629, 0.00117039, 0.00116895, ..., 0.2272452 , 0.00117119,
        0.00117006],
       [0.04886962, 0.0009863 , 0.15164142, ..., 0.03596778, 0.00098586,
        0.35984003],
       [0.1164423 , 0.00083975, 0.00084089, ..., 0.37960909, 0.00084059,
        0.22976872]])

In [21]:
arts['topic'] = np.argmax(arts_topics,axis=1)
arts['topic'].value_counts()

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0    3617
6    1696
1     837
7     561
3     294
5     238
4     107
2      55
Name: topic, dtype: int64

In [23]:
ids.to_csv('../patents/q2_topics.csv')
arts.to_csv('../papers/q2_topics.csv')